In [1]:
from numbers import Complex
import pandas as pd
import numpy as np
import os
import json
from tqdm import tqdm
import torch

In [2]:
data_dir = "/scratch/04703/sravula/UTAFSDataNew"

In [3]:
file_dirs = os.listdir(data_dir)
file_dirs = [f for f in file_dirs if ".pt" not in f]
print(len(file_dirs))

61


In [4]:
fname = os.path.join(data_dir, file_dirs[0])
print(fname)

/scratch/04703/sravula/UTAFSDataNew/case09


In [5]:
chip_num = int(fname.split("case")[-1])
print(chip_num)

9


In [6]:
df = pd.read_csv(os.path.join(fname, 'sparameters.csv'), sep=',')

In [7]:
df.head()

,Frequency,S11,S12,S13,S14,S21,S22,S23,S24,S31,S32,S33,S34,S41,S42,S43,S44
0,10.0,(0.0162895518+3.3658567e-08j),(0.983710447-1.47077344e-07j),(0.000541198288+1.2065333e-08j),(-0.000541198335-1.01989557e-08j),(0.983710447-1.47077344e-07j),(0.0162895528+6.36966957e-08j),(-0.000541198789-1.80003793e-08j),(0.00054119905+2.8302882e-08j),(0.000541198288+1.2065333e-08j),(-0.000541198789-1.80003793e-08j),(0.0162859277+4.44385684e-08j),(0.983714071-1.46876554e-07j),(-0.000541198335-1.01989557e-08j),(0.00054119905+2.8302882e-08j),(0.983714071-1.46876554e-07j),(0.0162859282+5.21219e-08j)
1,20020030.0,(0.0273069861+0.00197834442j),(0.951345625-0.198990628j),(0.00339866978+0.0111393863j),(-0.00110015345+0.000120446466j),(0.951345625-0.198990628j),(0.0268616643+0.00247440144j),(-0.00109024009+0.000123698099j),(0.00353580652+0.0110947919j),(0.00339866978+0.0111393863j),(-0.00109024009+0.000123698099j),(0.0272945297+0.00216531475j),(0.95139469-0.198836321j),(-0.00110015345+0.000120446466j),(0.00353580652+0.0110947919j),(0.95139469-0.198836321j),(0.0269166588+0.0026529526j)
2,40040050.0,(0.0292684287-5.231726e-05j),(0.889748483-0.385388636j),(0.00971315396+0.0205420918j),(-0.00108891754+0.000221126026j),(0.889748483-0.385388636j),(0.0277655374+0.00139156731j),(-0.00107703048+0.000225001448j),(0.0101988001+0.0202271826j),(0.00971315396+0.0205420918j),(-0.00107703048+0.000225001448j),(0.0293148349+0.000304627097j),(0.889898842-0.3851049j),(-0.00108891754+0.000221126026j),(0.0101988001+0.0202271826j),(0.889898842-0.3851049j),(0.02796406+0.00169292503j)
3,60060070.0,(0.0317898904-0.00271687759j),(0.792031883-0.555335596j),(0.0190229483+0.0266538883j),(-0.00106142861+0.00034493439j),(0.792031883-0.555335596j),(0.0287786335+0.000799283115j),(-0.00109009944+0.000366124925j),(0.0199685028+0.0258933604j),(0.0190229483+0.0266538883j),(-0.00109009944+0.000366124925j),(0.031864622-0.00212405084j),(0.792315414-0.554944138j),(-0.00106142861+0.00034493439j),(0.0199685028+0.0258933604j),(0.792315414-0.554944138j),(0.029232121+0.00102949912j)
4,80080090.0,(0.034632948-0.00625590192j),(0.662594875-0.701428715j),(0.0298776059+0.0286191782j),(-0.00106494743+0.000602395559j),(0.662594875-0.701428715j),(0.0302756885+9.50504529e-05j),(-0.000972125733+0.000477142026j),(0.0310563996+0.0271889717j),(0.0298776059+0.0286191782j),(-0.000972125733+0.000477142026j),(0.0346766681-0.00566241706j),(0.662998448-0.701085289j),(-0.00106494743+0.000602395559j),(0.0310563996+0.0271889717j),(0.662998448-0.701085289j),(0.0309089814+0.000245745772j)


In [8]:
num_ports = np.sqrt(len(df.columns) - 1)
is_int = np.floor(num_ports) == np.ceil(num_ports)

if not is_int:
    print("ERROR!")
else:
    num_ports = int(num_ports)
    print("NUM PORTS: ", num_ports)

NUM PORTS:  4


In [9]:
freq = df["Frequency"].to_numpy().astype(np.float32).squeeze()

In [10]:
print(freq.shape)
print(freq[0])
print(freq[1])

(1000,)
10.0
20020030.0


In [11]:
for i in range(num_ports):
    for j in range(i, num_ports):
        print("S" + str(i+1) + str(j+1))

S11
S12
S13
S14
S22
S23
S24
S33
S34
S44


In [12]:
df["S11"].head()

0    (0.0162895518+3.3658567e-08j)
1    (0.0273069861+0.00197834442j)
2     (0.0292684287-5.231726e-05j)
3    (0.0317898904-0.00271687759j)
4     (0.034632948-0.00625590192j)
Name: S11, dtype: object

In [13]:
val = df["S11"].values
val = np.asarray(val, dtype=complex)
val = np.stack((val.real, val.imag)).astype(np.float32)

In [14]:
print(val[:, 0])
print(val[:, 1])
print(val.shape)
print(np.max(val))
print(np.min(val))

[1.6289553e-02 3.3658566e-08]
[0.02730699 0.00197834]
(2, 1000)
0.087441556
-0.685601


In [15]:
#Want a dataset that's [1, 2, num_freqs, N, N]

# MAKE THE DATASET

In [16]:
data_dir = "/scratch/04703/sravula/UTAFSDataNew"

In [17]:
file_dirs = os.listdir(data_dir)
print(len(file_dirs))

63


In [18]:
file_dirs = [f for f in file_dirs if ".pt" not in f]
print(len(file_dirs))

61


In [19]:
frequencies = [None] * (len(file_dirs) + 1)
data = [None] * (len(file_dirs) + 1)

In [20]:
print(len(frequencies))
print(len(data))

62
62


In [21]:
for i, filename in tqdm(enumerate(file_dirs)):
    print(str(i) + ": " + filename)
    
    #grab the chip number
    chip_num = int(filename.split("case")[-1])
    print("CHIP NUM: ", chip_num)
    
    fname = os.path.join(data_dir, filename)
    
    #grab the dataframe
    df = pd.read_csv(os.path.join(fname, 'sparameters.csv'), sep=',')
    
    #validate the number of ports
    num_ports = np.sqrt(len(df.columns) - 1)
    is_int = np.floor(num_ports) == np.ceil(num_ports)

    if not is_int:
        print("ERROR!")
        break
    else:
        num_ports = int(num_ports)
        print("NUM PORTS: ", num_ports)
        
    #grab the frequency data
    freq = df["Frequency"].to_numpy().astype(np.float32).squeeze()
    num_freqs = len(freq)
    
    print("NUM FREQS: ", num_freqs)
    
    frequencies[chip_num] = torch.from_numpy(freq).clone()
    
    #grab the data
    num_unique = (num_ports + 1) * num_ports // 2
    print("NUM UNIQUE: ", num_unique)
    
    cur_data = torch.zeros(1, 2, num_freqs, num_unique)
    
    cur_idx = 0
    for i in range(num_ports):
        for j in range(i, num_ports):
            sparam = "S" + str(i+1) + str(j+1)
            
            val = df[sparam].values
            val = np.asarray(val, dtype=complex)
            val = np.stack((val.real, val.imag)).astype(np.float32)
            
            cur_data[0, :, :, cur_idx] = torch.from_numpy(val).clone()
            
            cur_idx = cur_idx + 1
    
    print("FINAL SHAPE: ", cur_data.shape, "\n")
    data[chip_num] = cur_data.clone()

0it [00:00, ?it/s]

0: case09
CHIP NUM:  9
NUM PORTS:  4
NUM FREQS:  1000
NUM UNIQUE:  10
FINAL SHAPE:  torch.Size([1, 2, 1000, 10]) 

1: case04
CHIP NUM:  4
NUM PORTS:  2
NUM FREQS:  1000
NUM UNIQUE:  3
FINAL SHAPE:  torch.Size([1, 2, 1000, 3]) 

2: case08
CHIP NUM:  8
NUM PORTS:  13
NUM FREQS:  500
NUM UNIQUE:  91


3it [00:00, 24.59it/s]

FINAL SHAPE:  torch.Size([1, 2, 500, 91]) 

3: case15
CHIP NUM:  15
NUM PORTS:  2
NUM FREQS:  1000
NUM UNIQUE:  3
FINAL SHAPE:  torch.Size([1, 2, 1000, 3]) 

4: case03
CHIP NUM:  3
NUM PORTS:  4
NUM FREQS:  101
NUM UNIQUE:  10
FINAL SHAPE:  torch.Size([1, 2, 101, 10]) 

5: case19
CHIP NUM:  19
NUM PORTS:  2
NUM FREQS:  341
NUM UNIQUE:  3
FINAL SHAPE:  torch.Size([1, 2, 341, 3]) 

6: case26
CHIP NUM:  26
NUM PORTS:  16
NUM FREQS:  9000
NUM UNIQUE:  136


7it [00:02,  2.12it/s]

...

FINAL SHAPE:  torch.Size([1, 2, 9999, 78]) 

59: case61
CHIP NUM:  61


60it [01:08,  1.08it/s]

NUM PORTS:  12
NUM FREQS:  2000
NUM UNIQUE:  78
FINAL SHAPE:  torch.Size([1, 2, 2000, 78]) 

60: case60
CHIP NUM:  60
NUM PORTS:  24
NUM FREQS:  9999
NUM UNIQUE:  300


61it [01:15,  1.23s/it]

FINAL SHAPE:  torch.Size([1, 2, 9999, 300]) 



In [22]:
torch.save(data, "/scratch/04703/sravula/UTAFSDataNew/NEW_DATA.pt")
torch.save(frequencies, "/scratch/04703/sravula/UTAFSDataNew/NEW_FREQUENCIES.pt")